# Data Set

*The FER2013 dataset used, is a popular facial expression recognition dataset used for training and evaluating emotion recognition models.*

*Link for the dataset* : **https://www.kaggle.com/datasets/msambare/fer2013/**

# Detection Code

*Importing the required dependencies*

In [3]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

In [6]:
train_dir = 'train/'
test_dir = 'test/'

*Defining Function to create data frames for both train and test directory*

In [8]:
def createdf(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "Labelled")
    return image_paths, labels

*Labelling the datasets - test and train*

In [9]:
train = pd.DataFrame()
train['image'], train['label'] = createdf(train_dir)

test = pd.DataFrame()
test['image'], test['label'] = createdf(test_dir)

angry Labelled
disgust Labelled
fear Labelled
happy Labelled
neutral Labelled
sad Labelled
surprise Labelled
angry Labelled
disgust Labelled
fear Labelled
happy Labelled
neutral Labelled
sad Labelled
surprise Labelled


In [11]:
def feat_ext(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img = img_to_array(img) / 255.0
        features.append(img)
    features = np.array(features)
    return features

In [12]:
train_feats = feat_ext(train['image'])
test_feats = feat_ext(test['image'])

  0%|          | 0/28709 [00:00<?, ?it/s]

  0%|          | 0/7178 [00:00<?, ?it/s]

*Using Label Encoder - To convert a class vector (integers) to binary class matrix.*

In [13]:
lbed = LabelEncoder()
lbed.fit(train['label'])
y_train = to_categorical(lbed.transform(train['label']), num_classes=7)
y_test = to_categorical(lbed.transform(test['label']), num_classes=7)

*Defining the Model Layers*

In [16]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation='softmax'))

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Defining callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model_weights.h5', save_best_only=True)
learning_rate_scheduler = LearningRateScheduler(lambda epoch: 0.001 * 0.9 ** epoch)

# Training the model
model.fit(x=train_feats, y=y_train, batch_size=128, epochs=100, validation_data=(test_feats, y_test), callbacks=[early_stop, model_checkpoint, learning_rate_scheduler])


Epoch 1/100
225/225 [==============================] - ETA: 0s - loss: 2.1298 - accuracy: 0.2124

C:\Users\prana\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


225/225 [==============================] - 1110s 5s/step - loss: 2.1298 - accuracy: 0.2124 - val_loss: 1.9109 - val_accuracy: 0.2471 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 958s 4s/step - loss: 1.7204 - accuracy: 0.3307 - val_loss: 2.2527 - val_accuracy: 0.2487 - lr: 9.0000e-04
Epoch 3/100
225/225 [==============================] - 877s 4s/step - loss: 1.5015 - accuracy: 0.4198 - val_loss: 1.9542 - val_accuracy: 0.3402 - lr: 8.1000e-04
Epoch 4/100
225/225 [==============================] - 848s 4s/step - loss: 1.3672 - accuracy: 0.4745 - val_loss: 1.3024 - val_accuracy: 0.4979 - lr: 7.2900e-04
Epoch 5/100
225/225 [==============================] - 852s 4s/step - loss: 1.2800 - accuracy: 0.5127 - val_loss: 1.2740 - val_accuracy: 0.5100 - lr: 6.5610e-04
Epoch 6/100
225/225 [==============================] - 849s 4s/step - loss: 1.2156 - accuracy: 0.5351 - val_loss: 1.2724 - val_accuracy: 0.5110 - lr: 5.9049e-04
Epoch 7/100
225/225 [============================


KeyboardInterrupt



***Interrupted the execution because too many epochs. 15-20 Epochs are more than enough. Keep Patience in range of 2-4***

In [19]:
# Save the model architecture to a JSON file
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)